<a href="https://colab.research.google.com/github/Byeon-MJ/DL_Computer_Vision/blob/main/Face_Tracking.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Face Tracking


## Tracking 하는 이유?
- Tracking이 Detection보다 처리속도가 빠르다
- 따라서 제일 처음에만 Detection하고 이후에는 Tracking을 해서 부드럽게 Tracking 하도록 구현

In [1]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [2]:
import cv2
import time

## Object Tracker mode 설정

In [20]:
# csrt
# tracker = cv2.TrackerCSRT_create()
# kcf
# tracker = cv2.TrackerKCF_create()
# boosting
# tracker = cv2.TrackerBoosting_create()
# mil
# tracker = cv2.TrackerMIL_create()
# tld
# tracker = cv2.TrackerTLD_create()
# medianflow
# tracker = cv2.TrackerMedianFlow_create()
# mosse
# tracker = cv2.TrackerMOSSE_create()

## OpenCV version 4.6.0
# csrt
# tracker = cv2.legacy.TrackerCSRT_create()
# kcf
tracker = cv2.legacy.TrackerKCF_create()
# boosting
# tracker = cv2.legacy.TrackerBoosting_create()
# mil
# tracker = cv2.legacy.TrackerMIL_create()
# tld
# tracker = cv2.legacy.TrackerTLD_create()
# medianflow
# tracker = cv2.legacy.TrackerMedianFlow_create()
# mosse
# tracker = cv2.legacy.TrackerMOSSE_create()

In [16]:
# 얼굴을 인식하는 haarcascade feature 파일
face_cascade_name = '/content/gdrive/MyDrive/CV/Face Detection/data/haarcascades/haarcascade_frontalface_alt.xml'
# face_cascade 객체 생성
face_cascade = cv2.CascadeClassifier()

if not face_cascade.load(cv2.samples.findFile(face_cascade_name)):
    print('### Error loading face cascade ###')
    exit(0)

In [17]:
# 원본 동영상
file_name = '/content/gdrive/MyDrive/CV/Face Tracking/mytracking.mp4'
# Tracking 된 output 동영상
output_name = 'face_tracking_video.mp4'

# 변수 초기화
frame_count = 0
detected = False
frame_mode = 'Tracking'
elapsed_time = 0

In [18]:
# 원본 동영상 로드
cap = cv2.VideoCapture(file_name)

# 원본 동영상 오픈 여부 확인
if not cap.isOpened:
    print('--(!) Error opening video capture')
    exit(0)

writer = None

# trackers 객체 생성
# trackers = cv2.MultiTracker_create()

# OvenCV 4.6.0 이후는 legacy에 들어가있음!! 버전 확인
trackers = cv2.legacy.MultiTracker_create()

In [21]:
while True:
    # 원본 동영상에서 frame을 읽는다
    ret, frame = cap.read()

    # 원본 동영상에서 더 이상 frame을 읽지 못하면 Exit
    if frame is None:
        # close the video file pointers
        cap.release()
        # close the writer point
        writer.release()
        print('--(!) No captured frame -- Break!')
        print(f'elapsed time {elapsed_time:.3f} seconds')
        break
    
    start_time = time.time()
    frame_count += 1

    if detected:
        frame_mode = 'Tracking'
        # trackers 객체가 tracking 하여 변경된 boxing 영역 위치 좌표를 업데이트
        (success, boxes) = trackers.update(frame)
        for box in boxes:
            (x, y, w, h) = [int(v) for v in box]
            # 위치 좌표에 box 그리기
            cv2.rectangle(frame, (x, y), (x + w, y + h), (0, 255, 0), 2)

    else:
        frame_mode = 'Detection'    # Detection mode
        frame_gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
        frame_gray = cv2.equalizeHist(frame_gray)
        faces = face_cascade.detectMultiScale(frame_gray)

        # largest_box 초기화
        largest_box = 0

        # detect 된 box 중에서 largest box를 찾는다.
        for (_x, _y, _w, _h) in faces:
            SZ = _w * _h
            if SZ > largest_box:
                largest_box = SZ
                x, y, w, h = int(_x), int(_y), int(_w), int(_h)
        
        # Detect 된 얼굴에 Boxing
        cv2.rectangle(frame, (x, y), (x + w, y + h), (0, 255, 255), 4)

        # trackers 객체가 설정된 tracker mode로 boxing된 영역 tracking
        trackers.add(tracker, frame, (x, y, w, h))
        detected = True # detected mode 변경

    # video를 disk에 output 하기위해 writer 초기화
    if writer is None and output_name is not None:
        fourcc = cv2.VideoWriter_fourcc(*"DIVX")
        writer = cv2.VideoWriter(output_name, fourcc, 30, (frame.shape[1], frame.shape[0]), True)

    # disk에 frame을 write
    if writer is not None:
        writer.write(frame)

    frame_time = time.time() - start_time
    elapsed_time += frame_time
    print(f'[{frame_mode} Frame {frame_count} time {frame_time}')

[Detection Frame 2 time 0.2579789161682129
[Tracking Frame 3 time 0.013372182846069336
[Tracking Frame 4 time 0.01699376106262207
[Tracking Frame 5 time 0.016389131546020508
[Tracking Frame 6 time 0.02208876609802246
[Tracking Frame 7 time 0.016111135482788086
[Tracking Frame 8 time 0.016364574432373047
[Tracking Frame 9 time 0.016912460327148438
[Tracking Frame 10 time 0.01854729652404785
[Tracking Frame 11 time 0.01687788963317871
[Tracking Frame 12 time 0.02005147933959961
[Tracking Frame 13 time 0.017368078231811523
[Tracking Frame 14 time 0.018110036849975586
[Tracking Frame 15 time 0.015408992767333984
[Tracking Frame 16 time 0.01590704917907715
[Tracking Frame 17 time 0.0167086124420166
[Tracking Frame 18 time 0.0166776180267334
[Tracking Frame 19 time 0.016388893127441406
[Tracking Frame 20 time 0.016605377197265625
[Tracking Frame 21 time 0.01674175262451172
[Tracking Frame 22 time 0.020749330520629883
[Tracking Frame 23 time 0.020924806594848633
[Tracking Frame 24 time 0.0182